In [ ]:
import numpy as np
import math
import ceo
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
import plotly.plotly as py       
import plotly.graph_objs as go
init_notebook_mode()
%pylab inline

# Purpose

This notebook describes the Zemax ray tracing model of CEO, where each surface location and orientation is given in the coordinate system of the previous surface

# Field definition
A field in defined with a `Source` object whose main parameters are:
* the photometric band: `R`,
* the rays box size in meter: `D` and pixel: `nPx`, meaning that the field rays is sampling a square area of `D` $\times$ `D` m$^2$ with `nPx` $\times$ `nPx` rays,
* `rays_origin` sets the origin location of the ray bundle: 
    * `rays_origin[0]` and `rays_origin[1]` are respectively the x and y coordinates of the intersection of the chief ray with the first surface,
    * `rays_origin[2]` sets the z-plane where the rays originate from.
    
Rays are always coming downwards along the z-axis towards the first surface.

In [ ]:
D = 30e-3
nPx = 7
src  = ceo.Source("R",rays_box_size=D,rays_box_sampling=nPx,rays_origin=[0.0,0.0,0.2])
print src.wavelength*1e6
xyz = [src.rays.coordinates.host()]

# Surface definition

There are 6 surfaces in the optical train:
* a coordinate break at 45 degree,
* a mirror,
* a coordinate break at 45 degree,
* the entrace surface of the lens,
* the exit surface of the lens,
* a dummy surface in the focal plane.

Each surface location (`origin`) and orientation (`euler_angles`) is given with respect to the coordinate system of the previous surface

In [ ]:
S = [ceo.Conic(0,1,origin=[0,0,0.1],euler_angles=[math.pi/4,0,0],refractive_index=0,coord_break=True),
     ceo.Conic(0,1,refractive_index=-1),
     ceo.Conic(0,1,euler_angles=[math.pi/4,0,0],refractive_index=0,coord_break=True),
     ceo.Conic(1./100e-3,1,origin=[0,0,50e-3],refractive_index=1.514846),
     ceo.Conic(-1./100e-3,1,origin=[0,0,10e-3],refractive_index=1.0),
     ceo.Conic(0,1,origin=[0,0,90e-3],refractive_index=1.0)]

The same surfaces from the Zemax converter:
* without a fold mirror

In [ ]:
src_material = {'formula': 15, 'c': [], 'name': ''}
S = [ceo.Conic(0.0, 1.0, origin=[0.0, 0.0, -0.0], material={'formula': 15, 'c': [], 'name': ''}, euler_angles=[0.0, 0.0, 0.0]),
     ceo.Conic(-10.0, 1.0, origin=[0.0, 0.0, -0.1], material={'formula': 2, 'c': [1.03961212, 0.00600069867, 0.231792344, 0.0200179144, 1.01046945, 103.560653, 0.0, 0.0, 0.0, 0.0], 'name': u'BK7'}, euler_angles=[0.0, 0.0, 0.0]),
     ceo.Conic(10.0, 1.0, origin=[0.0, 0.0, -0.01], material={'formula': 15, 'c': [], 'name': ''}, euler_angles=[0.0, 0.0, 0.0]),
     ceo.Conic(0.0, 1.0, origin=[0.0, 0.0, -0.09], material={'formula': 15, 'c': [], 'name': ''}, euler_angles=[0.0, 0.0, 0.0])]

* with a fold mirror

In [ ]:
src_material = {'formula': 15, 'c': [], 'name': ''}
S = [ceo.Conic(0.0, 1.0, origin=[0.0, 0.0, -0.0], material={'formula': 15, 'c': [], 'name': ''}, euler_angles=[0.0, 0.0, 0.0]),
     ceo.Conic(0.0, 1.0, origin=[0.0, 0.0, -0.1], coord_break=True, material={'formula': 15, 'c': [], 'name': ''},
               euler_angles=[math.pi/4, 0.0, 0.0]),
     ceo.Conic(0.0, 1.0, origin=[0.0, 0.0, -0.0], material={'formula': 14, 'c': [], 'name': u'MIRROR'}, euler_angles=[0.0, 0.0, 0.0]),
     ceo.Conic(-0.0, 1.0, origin=[0.0, 0.0, -0.0], coord_break=True, material={'formula': 15, 'c': [], 'name': ''},
               euler_angles=[math.pi/4, 0.0, 0.0]),
     ceo.Conic(10.0, 1.0, origin=[0.0, 0.0, 0.05], material={'formula': 2, 'c': [1.03961212, 0.00600069867, 0.231792344, 0.0200179144, 1.01046945, 103.560653, 0.0, 0.0, 0.0, 0.0], 'name': u'BK7'}, euler_angles=[0.0, 0.0, 0.0]),
     ceo.Conic(-10.0, 1.0, origin=[0.0, 0.0, 0.01], material={'formula': 15, 'c': [], 'name': ''}, euler_angles=[0.0, 0.0, 0.0]),
     ceo.Conic(0.0, 1.0, origin=[0.0, 0.0, 0.09], material={'formula': 15, 'c': [], 'name': ''}, euler_angles=[0.0, 0.0, 0.0])]

# Ray tracing

The routine below prints the chief ray surface intersection coordinates and unit direction vector:

In [ ]:
def chief():
    print 'XYZ: '+np.array_str(src.rays.chief_coordinates.host())
    print 'KLM: '+np.array_str(src.rays.chief_directions.host())

The rays are traced from a surface to the next surface following the same procedure for each surface:

1. The rays are transformed into the coordinate system of the surface,
2. If the surface is of the  coordinate break type, there is nothing else to do
3. If the surface is not of the  coordinate break type, the coordinates at the intersection of the rays with the surface are computed,
4. The following action depends on the refraction index `n` value:
    * `n=0`: there is nothing to do,
    * `n=-1`: the new direction vector of the rays are computed according to a reflection from the surface,
    * `n>0`: the new direction vector of the rays are computed according to a refraction through the surface and based on the ratio of the refractive indices of the materials before and after the surface; the current refraction index of the rays is update to the refractive index of the surface,

In [ ]:
def trace(S,idx):
    print 'Material refractive index: %.9f'%src.rays.refractive_index
    _S_ = S[idx-1]
    ceo.Transform_to_S(src,_S_)
    if not _S_.coord_break: 
        ceo.Intersect(src,_S_)
        n_S = _S_.refractive_index(src)
        if n_S==-1:
            ceo.Reflect(src)
        else:
            mu = src.rays.refractive_index/n_S
            if mu!=1.0:
                ceo.Refract(src,mu)
            src.rays.refractive_index = n_S
    
        #print 'To GCS:'
        for k in range(idx-1,-1,-1):
            #print k
            ceo.Transform_to_R(src,S[k])
        xyz.append( src.rays.coordinates.host())
        if idx<len(S):
            #print 'To last surface CS:'
            for k in range(idx):
                #print k
                ceo.Transform_to_S(src,S[k])
    chief()

The rays propagation is computed with

In [ ]:
[trace(S,k+1) for k in range(len(S))]

## Display

In [ ]:
def coords(ray_idx,xyz_idx):
    return [w[ray_idx,xyz_idx] for w in xyz]

In [ ]:
from plotly import tools
fig = tools.make_subplots(rows=2, cols=2, shared_yaxes=True)
h = []
for k in range(nPx**2):
    h = go.Scatter(x=coords(k,1),y=coords(k,2) ,
            mode='lines',showlegend=False, 
            line=dict(width=1,color='rgb(0.2,0.2,0.2'),opacity=0.1 )
    fig.append_trace(h, 1, 1)
    h = go.Scatter(x=coords(k,0),y=coords(k,2) ,
            mode='lines',showlegend=False, 
            line=dict(width=1,color='rgb(0.2,0.2,0.2'),opacity=0.1 )
    fig.append_trace(h, 1, 2)
#data = go.Data(h)
layout = go.Layout(width=500,height=500)
#fig = go.Figure(data=data,layout=layout)
iplot(fig)

In [ ]:
print xyz[0][0,:]
print coords(0,1)
print coords(0,2)

## Spot diagram

In [ ]:
h = go.Scatter(x=xyz[-1][:,0],y=0.1-xyz[-1][:,2],mode='markers')
layout = go.Layout(width=500,height=500)
iplot(go.Figure(data=go.Data([h]),layout=layout))